In [4]:
#!pip3.10 install pyreadstat

zsh:1: command not found: pip3.9


# Example of importing data statistics from Stata
- Extract data dictionary and data from Stata file
- Apply weights to one or more variables
- Set user defined missings
- Export selective or all variables to NADA/MetadataEditor compatible format


In [1]:
import pyreadstat
import time
import re
import pandas as pd
import numpy as np
import os
from src.FileInfo import FileInfo
from src.VarInfo import VarInfo
from src.DictParams import DictParams
from src.DataUtils import DataUtils
from src.DataDictionary import DataDictionary
from statsmodels.stats.weightstats import DescrStatsW




In [3]:
# file is available at https://github.com/thousandoaks/Wooldridge/blob/master/lawsch85.dta
filepath="/Volumes/webdev/editor/datafiles/editor/0c74b7f78409a4022a2c4c5a5ca3ee19/data/LAWSCH85.DTA"

# output CSV file path
csvfile="/users/m2/Downloads/sample_data.csv"

datadict=DataDictionary()
fileinfo= FileInfo(file_path=filepath)



In [4]:
# get metadata for variables witout summary statistics
metadata=datadict.get_metadata(fileinfo)

In [6]:
metadata

{'rows': 156,
 'columns': 21,
 'variables': [{'name': 'rank',
   'labl': 'law school ranking',
   'var_intrvl': 'contin',
   'var_format': {'type': 'numeric', 'schema': 'other'},
   'var_catgry_labels': []},
  {'name': 'salary',
   'labl': 'median starting salary',
   'var_intrvl': 'contin',
   'var_format': {'type': 'numeric', 'schema': 'other'},
   'var_catgry_labels': []},
  {'name': 'cost',
   'labl': 'law school cost',
   'var_intrvl': 'contin',
   'var_format': {'type': 'numeric', 'schema': 'other'},
   'var_catgry_labels': []},
  {'name': 'LSAT',
   'labl': 'median LSAT score',
   'var_intrvl': 'contin',
   'var_format': {'type': 'numeric', 'schema': 'other'},
   'var_catgry_labels': []},
  {'name': 'GPA',
   'labl': 'median college GPA',
   'var_intrvl': 'contin',
   'var_format': {'type': 'numeric', 'schema': 'other'},
   'var_catgry_labels': []},
  {'name': 'libvol',
   'labl': 'no. volumes in lib., 1000s',
   'var_intrvl': 'contin',
   'var_format': {'type': 'numeric', 'sche

In [7]:
# get data dictionary with all summary statistics
variable_dict=datadict.get_data_dictionary(fileinfo)

In [8]:
variable_dict

{'rows': 156,
 'columns': 21,
 'variables': [{'name': 'rank',
   'labl': 'law school ranking',
   'var_intrvl': 'contin',
   'loc_width': 9,
   'var_valrng': {'range': {'UNITS': 'REAL',
     'count': 156,
     'min': '1.0',
     'max': '175.0'}},
   'var_sumstat': [{'type': 'vald', 'value': '156'},
    {'type': 'invd', 'value': '0'},
    {'type': 'min', 'value': '1.0'},
    {'type': 'max', 'value': '175.0'},
    {'type': 'mean', 'value': '83.75'},
    {'type': 'stdev', 'value': '50.064168501453565'}],
   'var_catgry': [],
   'var_catgry_labels': [],
   'var_format': {'type': 'numeric', 'schema': 'other'}},
  {'name': 'salary',
   'labl': 'median starting salary',
   'var_intrvl': 'contin',
   'loc_width': 9,
   'var_valrng': {'range': {'UNITS': 'REAL',
     'count': 148,
     'min': '24900.0',
     'max': '78325.0'}},
   'var_sumstat': [{'type': 'vald', 'value': '148'},
    {'type': 'invd', 'value': '8'},
    {'type': 'min', 'value': '24900.0'},
    {'type': 'max', 'value': '78325.0'},

In [7]:
params=DictParams(
    file_path=filepath,
    weights=[
            {
                "field": "top10",
                "weight_field": "llibvol"
            }
        ]    
)

dict2=datadict.get_data_dictionary_variable(params)

weights:  {'top10': {'wgt_freq': {0: 826, 1: 64}, 'wgt_mean': 0.0724168303083735, 'wgt_stdev': 0.25932231036778775}}
field= top10 value= 0
freq_values {0: 826, 1: 64}
field= top10 value= 1
freq_values {0: 826, 1: 64}


In [10]:
dict2

{'rows': 156,
 'columns': 21,
 'variables': [{'name': 'rank',
   'labl': 'law school ranking',
   'var_intrvl': 'contin',
   'loc_width': 9,
   'var_valrng': {'range': {'UNITS': 'REAL',
     'count': 156,
     'min': '1.0',
     'max': '175.0'}},
   'var_sumstat': [{'type': 'vald', 'value': '156'},
    {'type': 'invd', 'value': '0'},
    {'type': 'min', 'value': '1.0'},
    {'type': 'max', 'value': '175.0'},
    {'type': 'mean', 'value': '83.75'},
    {'type': 'stdev', 'value': '50.064168501453565'}],
   'var_catgry': [],
   'var_catgry_labels': [],
   'var_format': {'type': 'numeric', 'schema': 'other'}},
  {'name': 'salary',
   'labl': 'median starting salary',
   'var_intrvl': 'contin',
   'loc_width': 9,
   'var_valrng': {'range': {'UNITS': 'REAL',
     'count': 148,
     'min': '24900.0',
     'max': '78325.0'}},
   'var_sumstat': [{'type': 'vald', 'value': '148'},
    {'type': 'invd', 'value': '8'},
    {'type': 'min', 'value': '24900.0'},
    {'type': 'max', 'value': '78325.0'},

In [9]:
# read metadata and data from Stata file
df,meta = pyreadstat.read_dta(fileinfo.file_path)

In [17]:
# print metadata (column names, types, labels)
meta.column_names_to_labels

{'rank': 'law school ranking',
 'salary': 'median starting salary',
 'cost': 'law school cost',
 'LSAT': 'median LSAT score',
 'GPA': 'median college GPA',
 'libvol': 'no. volumes in lib., 1000s',
 'faculty': 'no. of faculty',
 'age': 'age of law sch., years',
 'clsize': 'size of entering class',
 'north': '=1 if law sch in north',
 'south': '=1 if law sch in south',
 'east': '=1 if law sch in east',
 'west': '=1 if law sch in west',
 'lsalary': 'log(salary)',
 'studfac': 'student-faculty ratio',
 'top10': '=1 if ranked in top 10',
 'r11_25': '=1 if ranked 11-25',
 'r26_40': '=1 if ranked 26-40',
 'r41_60': '=1 if ranked 41-60',
 'llibvol': 'log(libvol)',
 'lcost': 'log(cost)'}

In [25]:
# column names
meta.column_names

['rank',
 'salary',
 'cost',
 'LSAT',
 'GPA',
 'libvol',
 'faculty',
 'age',
 'clsize',
 'north',
 'south',
 'east',
 'west',
 'lsalary',
 'studfac',
 'top10',
 'r11_25',
 'r26_40',
 'r41_60',
 'llibvol',
 'lcost']

In [26]:
# column data types info
meta.original_variable_types

{'rank': '%9.0g',
 'salary': '%9.0g',
 'cost': '%9.0g',
 'LSAT': '%9.0g',
 'GPA': '%9.0g',
 'libvol': '%9.0g',
 'faculty': '%9.0g',
 'age': '%9.0g',
 'clsize': '%9.0g',
 'north': '%9.0g',
 'south': '%9.0g',
 'east': '%9.0g',
 'west': '%9.0g',
 'lsalary': '%9.0g',
 'studfac': '%9.0g',
 'top10': '%9.0g',
 'r11_25': '%9.0g',
 'r26_40': '%9.0g',
 'r41_60': '%9.0g',
 'llibvol': '%9.0g',
 'lcost': '%9.0g'}

In [22]:
# Pyreadstat offers many other metadata options, see https://github.com/Roche/pyreadstat?tab=readme-ov-file

In [13]:
# print data
df

,rank,salary,cost,LSAT,GPA,libvol,faculty,age,clsize,north,...,east,west,lsalary,studfac,top10,r11_25,r26_40,r41_60,llibvol,lcost
0,128,31400.0,8340,155,3.15,216,45,12,210,1,...,0,0,10.354563,4.666667,0,0,0,0,5.375278,9.028818
1,104,33098.0,6980,160,3.50,256,44,113,190,0,...,0,0,10.407228,4.318182,0,0,0,0,5.545177,8.850804
2,34,32870.0,16370,155,3.25,424,78,134,270,0,...,1,0,10.400315,3.461539,0,0,1,0,6.049734,9.703206
3,49,35000.0,17566,157,3.20,329,136,89,277,0,...,1,0,10.463103,2.036765,0,0,0,1,5.796058,9.773721
4,95,33606.0,8350,162,3.38,332,56,70,150,0,...,0,1,10.422460,2.678571,0,0,0,0,5.805135,9.030017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,17,49321.0,13530,162,3.34,300,50,206,178,0,...,1,0,10.806106,3.560000,0,1,0,0,5.703783,9.512665
152,21,49900.0,11334,161,3.40,NaN,47,NaN,285,1,...,0,0,10.817777,6.063830,0,1,0,0,NaN,9.335563
153,143,31500.0,7396,157,3.40,174,17,65,79,0,...,0,1,10.357743,4.647059,0,0,0,0,5.159055,8.908694
154,3,69000.0,19780,171,3.82,850,100,140,101,0,...,1,0,11.141862,1.010000,1,0,0,0,6.745236,9.892426


In [8]:
# export data to stata version 8
#pyreadstat.pyreadstat.write_dta(df, "/Downloads/export-stata-8.dta", version=8)

In [27]:
# get df columns with type info
df.dtypes

rank         int64
salary     float64
cost        object
LSAT        object
GPA        float64
libvol      object
faculty     object
age         object
clsize      object
north        int64
south        int64
east         int64
west         int64
lsalary    float64
studfac    float64
top10        int64
r11_25       int64
r26_40       int64
r41_60       int64
llibvol    float64
lcost      float64
dtype: object

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 21 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   rank     156 non-null    int64  
 1   salary   148 non-null    float64
 2   cost     150 non-null    object 
 3   LSAT     150 non-null    object 
 4   GPA      149 non-null    float64
 5   libvol   155 non-null    object 
 6   faculty  152 non-null    object 
 7   age      111 non-null    object 
 8   clsize   153 non-null    object 
 9   north    156 non-null    int64  
 10  south    156 non-null    int64  
 11  east     156 non-null    int64  
 12  west     156 non-null    int64  
 13  lsalary  148 non-null    float64
 14  studfac  150 non-null    float64
 15  top10    156 non-null    int64  
 16  r11_25   156 non-null    int64  
 17  r26_40   156 non-null    int64  
 18  r41_60   156 non-null    int64  
 19  llibvol  155 non-null    float64
 20  lcost    150 non-null    float64
dtypes: float64(6), i

In [3]:
# example for apply weights
# uses WeightsColumn dict:
# {"field": "f1", "weight_field": "fld_weight"}
# where f1 is the field to be weighted and fld_weight is the field with the weights

# var_names - list of variables to return

# init params
params_with_wgt= DictParams(file_path=filepath, var_names=["GPA"], weights=[{"weight_field":"lcost", "field":"GPA"}])

In [4]:
params_with_wgt

DictParams(file_path='/Volumes/webdev/editor/datafiles/editor/0c74b7f78409a4022a2c4c5a5ca3ee19/data/LAWSCH85.DTA', var_names=['GPA'], weights=[WeightsColumns(field='GPA', weight_field='lcost')], missings={}, dtypes={}, value_labels={}, name_labels={}, export_format='csv')

In [5]:
# get data with weights for a single variable - GPA
variable_dict_wgt=datadict.get_data_dictionary_variable(params_with_wgt)

column name GPA
column name lcost
weights:  {'GPA': {'wgt_freq': {2: 9, 3: 9}, 'wgt_mean': 3.31224655102333, 'wgt_stdev': 0.19451495783479375}}


In [6]:
variable_dict_wgt

{'rows': 156,
 'columns': 2,
 'variables': [{'name': 'GPA',
   'labl': 'median college GPA',
   'var_intrvl': 'contin',
   'loc_width': 9,
   'var_valrng': {'range': {'UNITS': 'REAL',
     'count': 149,
     'min': '2.7300000190734863',
     'max': '3.819999933242798'}},
   'var_sumstat': [{'type': 'vald', 'value': '149'},
    {'type': 'invd', 'value': '7'},
    {'type': 'min', 'value': '2.7300000190734863'},
    {'type': 'max', 'value': '3.819999933242798'},
    {'type': 'mean', 'value': '3.302751677148294'},
    {'type': 'stdev', 'value': '0.19977642750979446'},
    {'type': 'mean', 'wgtd': 'wgtd', 'value': 3.31224655102333},
    {'type': 'stdev', 'wgtd': 'wgtd', 'value': 0.19451495783479375}],
   'var_catgry': [],
   'var_catgry_labels': [],
   'var_format': {'type': 'numeric', 'schema': 'other'}},
  {'name': 'lcost',
   'labl': 'log(cost)',
   'var_intrvl': 'contin',
   'loc_width': 9,
   'var_valrng': {'range': {'UNITS': 'REAL',
     'count': 150,
     'min': '7.872074127197266',


In [7]:
# example for setting user defined missings
#dictparams2= DictParams(file_path=filepath, 
#                        weights=[{"weight_field":"a1", "field":"a2"}], 
#                        missings=[{"field":"a1","missings":[1,"2"]}])

missing_params={        
        "age":[206]
    }


# set age=206 as missing
params_with_missing= DictParams(file_path=filepath, var_names=["age"], missings=missing_params)
params_with_missing

DictParams(file_path='/Volumes/webdev/editor/datafiles/editor/0c74b7f78409a4022a2c4c5a5ca3ee19/data/LAWSCH85.DTA', var_names=['age'], weights=[], missings={'age': [206]}, dtypes={}, value_labels={}, name_labels={}, export_format='csv')

In [8]:
result_miss=datadict.get_data_dictionary_variable(params_with_missing)
result_miss

{'rows': 156,
 'columns': 1,
 'variables': [{'name': 'age',
   'labl': 'age of law sch., years',
   'var_intrvl': 'discrete',
   'loc_width': 9,
   'var_valrng': {'range': {'UNITS': 'REAL',
     'count': 111,
     'min': '3.0',
     'max': '206.0'}},
   'var_sumstat': [{'type': 'vald', 'value': '110'},
    {'type': 'invd', 'value': '46'},
    {'type': 'min', 'value': '3.0'},
    {'type': 'max', 'value': '196.0'},
    {'type': 'mean', 'value': '84.72727272727273'},
    {'type': 'stdev', 'value': '39.41775995453311'}],
   'var_catgry': [{'value': '3', 'stats': [{'type': 'freq', 'value': '1'}]},
    {'value': '6', 'stats': [{'type': 'freq', 'value': '1'}]},
    {'value': '9', 'stats': [{'type': 'freq', 'value': '1'}]},
    {'value': '10', 'stats': [{'type': 'freq', 'value': '1'}]},
    {'value': '12', 'stats': [{'type': 'freq', 'value': '3'}]},
    {'value': '13', 'stats': [{'type': 'freq', 'value': '1'}]},
    {'value': '20', 'stats': [{'type': 'freq', 'value': '1'}]},
    {'value': '23'